In [9]:
""" 
author:-aam35
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf

import time
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
import tensorflow as tf
import tensorflow_datasets as tfds





def load_mnist(path, kind='train', val_size = 0.0):
    import os
    import gzip
    import numpy as np
    from sklearn.model_selection import train_test_split

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)
    # if kind=='train':
    #     return train_test_split(images, labels, test_size=val_size, random_state=13)
    # else:
    img = images.reshape([-1, 28, 28, 1])
    return img, labels


# Define paramaters for the model
learning_rate = 0.001
batch_size = 32
n_epochs = 10
n_train = None
n_test = None
n_classes = 10
n_channel = 1
pixel_depth = 255
image_size = 28
seed = None

# Step 1: Read in data
fmnist_folder = './data/'
train_data, train_label= load_mnist('./data/')
test_data, test_label = load_mnist('./data/', kind='t10k')
#Create train,validation,test split
#train, val, test = utils.read_fmnist(fmnist_folder, flatten=True)

# train_data = tfds.load('fashion_mnist', tfds.Split.TRAIN)
# eval_data = tfds.load('fashion_mnist', tfds.Split.VALIDATION)
# test_data = tfds.load('fashion_mnist', tfds.Split.TEST)
# Step 2: Create datasets and iterator
# create training Dataset and batch it

# SHUFFLE_BUFFER_SIZE = 100

train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label)).batch(batch_size)
# val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label)).batch(batch_size)

# create testing Dataset and batch it


# train_dataset = train_dataset.batch_and_drop_remainder(BATCH_SIZE)
# # val_dataset = val_dataset.batch(BATCH_SIZE)
# test_dataset = test_dataset.batch_and_drop_remainder(BATCH_SIZE)

# create one iterator and initialize it with different datasets

# train_iterator = train_dataset[0:63]
# train_img, train_label = train_iterator.get_next()
#
# train_init = iterator.make_initializer(train_data)	# initializer for train_data
# test_init = iterator.make_initializer(test_data)	# initializer for train_data

# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y
# weight_initializer = tf.random_normal(mean=0.0, stddev=0.01)
# step 2: create the weight variable with proper initialization
# w = tf.get_variable(name="Weight", dtype=tf.float32, shape=(train_data.shape[1],train_data.shape[0]), initializer=weight_initializer)

# initialize biases as zero
# step 1: create the initializer for biases
# b =tf.constant(0., shape=train_label.shape, dtype=tf.float32)
#############################
########## TO DO ############
#############################

conv1_weights = tf.Variable(
    tf.random.normal([5, 5, n_channel, 32],  # 5x5 filter, depth 32.
                        stddev=0.1,
                        seed=seed, dtype=tf.float32))
conv1_biases = tf.Variable(tf.zeros([32], dtype=tf.float32))
conv2_weights = tf.Variable(tf.random.normal(
    [5, 5, 32, 64], stddev=0.1,
    seed=seed, dtype=tf.float32))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64], dtype=tf.float32))
fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.random.normal([image_size // 4 * image_size // 4 * 64, 512],
                        stddev=0.1,
                        seed=seed,
                        dtype=tf.float32))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512], dtype=tf.float32))
fc2_weights = tf.Variable(tf.random.normal([512, n_classes],
                                              stddev=0.1,
                                              seed=seed,
                                              dtype=tf.float32))
fc2_biases = tf.Variable(tf.constant(
    0.1, shape=[n_classes], dtype=tf.float32))




# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
def model(data, train=False):
    """The Model definition."""
    # 2D convolution, with 'SAME' padding (i.e. the output feature map has
    # the same size as the input). Note that {strides} is a 4D array whose
    # shape matches the data layout: [image index, y, x, depth].
    data = tf.cast(data, tf.float32)
    conv = tf.nn.conv2d(data,
                        conv1_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    conv = tf.nn.conv2d(pool,
                        conv2_weights,
                        strides=[1, 1, 1, 1],
                        padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(relu,
                          ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1],
                          padding='SAME')
    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(
        pool,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    # Add a 50% dropout during training only. Dropout also scales
    # activations such that no rescaling is needed at evaluation time.
    if train:
        hidden = tf.nn.dropout(hidden, 0.5, seed=seed)
    return tf.matmul(hidden, fc2_weights) + fc2_biases


#############################
########## TO DO ############
#############################


# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function

def loss_func(logit, y):
    return tf.compat.v2.nn.softmax_cross_entropy_with_logits(labels=tf.reshape(y,(batch_size, 1)), logits=logit)


def train(inputs, targets):
    loss_value = loss_func(inputs, targets)
    return loss_value

iter = train_dataset.__iter__()
train_img, train_lbl = iter.next()
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)

for i in range(n_epochs):
    for j in range(train_data.shape[0]//batch_size):
        with tf.GradientTape() as tape:
            logits = model(train_img, train=True)
            loss_value = train(logits, train_lbl)

        dc1w, dc1b ,dc2w, dc2b, df1w, df1b, df2w, df2b= tape.gradient(loss_value, [conv1_weights, conv1_biases,
                                                                                    conv2_weights, conv2_biases,
                                                                                    fc1_weights, fc1_biases,
                                                                                    fc2_weights, fc2_biases,])
        optimizer.apply_gradients(zip([dc1w, dc1b ,dc2w, dc2b, df1w, df1b, df2w, df2b],
                                      [conv1_weights, conv1_biases,
                                       conv2_weights, conv2_biases,
                                       fc1_weights, fc1_biases,
                                       fc2_weights, fc2_biases, ]))

In [10]:
itert = test_dataset.__iter__()
test_img, test_lbl = iter.next()
logits = model(test_img, train=False)

In [14]:
preds = tf.nn.softmax(logits)
preds

<tf.Tensor: id=3398109, shape=(32, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       

In [12]:
correct_preds = tf.equal(tf.argmax(preds, axis=-1), tf.argmax(test_lbl, axis=-1))
correct_preds

<tf.Tensor: id=3398108, shape=(32,), dtype=bool, numpy=
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])>

In [13]:
logits

<tf.Tensor: id=3398102, shape=(32, 10), dtype=float32, numpy=
array([[7.07752336e+17, 7.07713578e+17, 7.07625480e+17, 7.07765805e+17,
        7.07771715e+17, 7.07703683e+17, 7.07693238e+17, 7.07755085e+17,
        7.07735981e+17, 7.07814390e+17],
       [1.38965969e+17, 1.38958358e+17, 1.38941067e+17, 1.38968632e+17,
        1.38969783e+17, 1.38956400e+17, 1.38954355e+17, 1.38966519e+17,
        1.38962748e+17, 1.38978149e+17],
       [1.97826881e+17, 1.97816023e+17, 1.97791422e+17, 1.97830626e+17,
        1.97832310e+17, 1.97813274e+17, 1.97810354e+17, 1.97827637e+17,
        1.97822311e+17, 1.97844233e+17],
       [4.07238629e+17, 4.07216329e+17, 4.07165648e+17, 4.07246359e+17,
        4.07249761e+17, 4.07210625e+17, 4.07204578e+17, 4.07240209e+17,
        4.07229180e+17, 4.07274294e+17],
       [4.07498491e+17, 4.07476192e+17, 4.07425442e+17, 4.07506291e+17,
        4.07509624e+17, 4.07470454e+17, 4.07464441e+17, 4.07500037e+17,
        4.07489008e+17, 4.07534225e+17],
       [4.678

<tf.Tensor: id=566550, shape=(32,), dtype=bool, numpy=
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])>